#### A 2 node experiment to demonstrate the network simulation and attack process in cyborg

Scenario: In this scenario, there are two nodes (1 router node ) and other host node (user0). There is only one agent (red agent). By rule of the simulation, red agent should start with some observed host and can only attack on host not on routers. Since we have only one host, the red agent can observe it at the starting of game. However,red agent only knows the ip of host, processid, session and system level information. It doesnot know about open ports and the vulnerabilities in the host. 

In this experiment, the red agent: 
1. Discover the open ports on the host 
2. Deploy the exploits to the host to find out applicable escalate action if any, as well as processes that are required to be genuine.

In [1]:
import random
from os.path import dirname
from pprint import pprint
import inspect
from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator
from CybORG.Simulator.Actions import *


path = str(inspect.getfile(CybORG))
path= dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario_2nodes.yaml'

sg = FileReaderScenarioGenerator(path)
print('Scenario file path is:',path)

env = CybORG(scenario_generator=sg)
#pprint(dir(env))

#pprint('IP map of network is:',env.get_ip_map())
#pprint('Red agent state is:',env.get_agent_state('Red'))
#pprint('Red agent action space is:',env.get_action_space('Red'))
print('==> IP map of network is:')
pprint(env.get_ip_map())



ips= env.get_ip_map()
userip= ips['User0']
print('-> user ip to attack is:',userip)





#results = env.reset(agent='Red')
#agent = B_lineAgent()

#for step in range(30):
#    action = agent.get_action(results.observation,results.action_space)
#    #print('Action is:',action)
#    results = cyborg.step(action=action, agent='Red')
#    #print(results.reward) 

Scenario file path is: c:\users\hpp\desktop\cyborg\cyborg\CybORG/Simulator/Scenarios/scenario_files/Scenario_2nodes.yaml
==> IP map of network is:
{'User0': IPv4Address('10.0.83.222'), 'User_router': IPv4Address('10.0.83.220')}
-> user ip to attack is: 10.0.83.222


C:\Users\HPP\miniconda3\envs\cyborg\lib\site-packages\gym\utils\seeding.py:48: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  "Function `rng.randint(low, [high, size, dtype])` is marked as deprecated "


##### Observed state from the perspective of red agent

In [2]:
print('==> Red Agents observed state  is:')
pprint(env.get_agent_state('Red'))

==> Red Agents observed state  is:
{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.83.222'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.83.208/28')}],
           'Processes': [{'PID': 3375, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 0,
                         'PID': 3375,
                         'Timeout': 0,
                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                         'Username': 'SYSTEM'}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User0',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.WINDOWS: 2>,
                           'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>,
                           'position': array([0., 0.])}},
 'success': <Trinary

#### Action space to the red agent:

In [3]:
print('==> Red Agent action space is:')
pprint(env.get_action_space('Red'))


==> Red Agent action space is:
{'action': {<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True},
 'agent': {'Red': True},
 'hostname': {'User0': True, 'User_router': False},
 'ip_address': {IPv4Address('10.0.83.220'): False,
                IPv4Address('10.0.83.222'): True},
 'password': {'vagrant': False},
 'port': {21: False, 22: False},
 'process': {3344: False, 3368: False, 3375: True},
 'session': {0: True},
 'subnet': {IPv4Network('10.0.83.208/28'): True},
 'target_session': {0: True,
                    1: False,
                    2: False,
                    3: False,
                    4: False,
                    5: False,
        

#### Attack action : DiscoverRemoteServices
It allow the red agent to identify services on a host as a prerequisite for running an exploit.
It Calls the low level action 'PortScan' then modifies the observation. The outcome is all open ports running on the host.  It must be used on a host to successfully run the high level action ExploitRemoteServices. 


In [4]:
#creating and executing port discovery attack on user network 
action = DiscoverNetworkServices(session=0,agent='Red',ip_address=userip)
results = env.step(action=action,agent='Red')
print('==> Red agent state after discover network services attack:')
pprint(results.observation)

==> Red agent state after discover network services attack:
{'10.0.83.222': {'Interface': [{'IP Address': IPv4Address('10.0.83.222')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.83.222'),
                                                 'local_port': 22}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.83.222'),
                                                 'local_port': 21}]}]},
 'success': <TrinaryEnum.TRUE: 1>}


#### Attack action : ExploitRemoteServices
It allow the red agent to examines the target host and returns a selected applicable escalate action if any, as well as processes that are required to be genuine.

In [18]:
#creating and executing attack on user network (can we attack on router? -No)
action = ExploitRemoteService(ip_address=userip,session=0,agent='Red')
results = env.step(action=action,agent='Red')
print('==> Red agent state after exploit attack:')
pprint(results.observation)

==> Red agent state after exploit attack:
{'10.0.8.233': {'Interface': [{'IP Address': IPv4Address('10.0.8.233')}],
                'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                'local_address': IPv4Address('10.0.8.233'),
                                                'local_port': 22}],
                               'Process Type': <ProcessType.SSH: 2>},
                              {'Connections': [{'local_address': IPv4Address('10.0.8.233'),
                                                'local_port': 50495,
                                                'remote_address': IPv4Address('10.0.8.233'),
                                                'remote_port': 22}]},
                              {'Connections': [{'local_address': IPv4Address('10.0.8.233'),
                                                'local_port': 22,
                                                'remote_address': IPv4Address('10.0.8.233'

In [19]:
## to do : Imapct and many things etc... 